In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.contrib.layers import fully_connected

In [2]:
#set hyperparameters

In [3]:
alpha=0.001
drop_prob = 0.5
sessMode  = "predict"
load_path = "old"
save_path = "old"

In [4]:
#construct the graph

In [5]:
Prod_size  =10
user_size  =10
brand_size =10

prod_embedding_size  =5
user_embedding_size  =10
brand_embedding_size =2

max_step_num    = 2
static_feat_dim = 2
n_neurons       = 10

n_output        = 10

In [6]:
prod_embeddings  = tf.Variable(tf.random_uniform([Prod_size,prod_embedding_size], -1.0, 1.0))
user_embeddings  = tf.Variable(tf.random_uniform([user_size,user_embedding_size], -1.0, 1.0))
brand_embeddings = tf.Variable(tf.random_uniform([brand_size,brand_embedding_size], -1.0, 1.0))

In [7]:
prod_idx  = tf.placeholder(tf.int32, shape=[None, max_step_num])
user_idx  = tf.placeholder(tf.int32, shape=[None, max_step_num])
brand_idx = tf.placeholder(tf.int32, shape=[None, max_step_num])

seq_length = tf.placeholder(tf.int32, [None])

static_feat = tf.placeholder(tf.float32, shape=[None, max_step_num,static_feat_dim])
prod_embed  = tf.nn.embedding_lookup(prod_embeddings , prod_idx)
user_embed  = tf.nn.embedding_lookup(user_embeddings , user_idx)
brand_embed = tf.nn.embedding_lookup(brand_embeddings, brand_idx)

y = tf.placeholder(tf.int32,[None])

In [8]:
final_feat=tf.concat([static_feat,prod_embed,user_embed,brand_embed], 2) 

In [9]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
if sessMode=="init_train" or sessMode=="continue_train":
    basic_cell = tf.contrib.rnn.DropoutWrapper(basic_cell,input_keep_prob=drop_prob)

In [10]:
outputs, states = tf.nn.dynamic_rnn(basic_cell, final_feat, dtype=tf.float32,sequence_length=seq_length)
logits = fully_connected(states, n_output, activation_fn=None)

In [11]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
loss = tf.reduce_mean(xentropy)

In [12]:
optimizer = tf.train.AdamOptimizer(learning_rate=alpha)
training_op = optimizer.minimize(loss)

In [13]:
correct = tf.nn.in_top_k(logits,y,1)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

In [14]:
init = tf.global_variables_initializer()

In [15]:
#feed data and train

In [16]:
prod_batch = np.array([
[1,2], # instance 0
[2,0], # instance 1 (padded with a zero vector)
[3,2], # instance 2
[1,1], # instance 3
])
user_batch = np.array([
[1,2], # instance 0
[2,0], # instance 1 (padded with a zero vector)
[3,2], # instance 2
[1,1], # instance 3
])
brand_batch = np.array([
[1,2], # instance 0
[2,0], # instance 1 (padded with a zero vector)
[3,2], # instance 2
[1,1], # instance 3
])

static_batch = np.array([
# step 0 step 1
[[1,2], [2,3]], # instance 0
[[3,5], [0,0]], # instance 1 (padded with a zero vector)
[[4,2], [1,3]], # instance 2
[[6,6], [7,1]], # instance 3
])

y_batch = np.array([2,1,3,8])
seq_length_batch = np.array([2, 2, 2, 2])

In [17]:
saver = tf.train.Saver()

In [18]:
n_epochs=100
batch_size=100

In [19]:
with tf.Session() as sess:
    if sessMode=="init_train":
        init.run()
        for epoch in range(n_epochs):
            nt,accuracy_eval = sess.run([training_op,accuracy], feed_dict={
                    prod_idx: prod_batch,user_idx:user_batch,brand_idx:brand_batch,static_feat:static_batch, 
                    seq_length: seq_length_batch,y:y_batch})
            print(accuracy_eval)
        save_path = saver.save(sess,"/Users/snakepointid/Documents/project/RProj/my code/r相关函数/save/"+save_path+"/test.ckpt")
    elif sessMode=="continue_train":
        saver.restore(sess,"/Users/snakepointid/Documents/project/RProj/my code/r相关函数/save/"+load_path+"/test.ckpt")
        for epoch in range(n_epochs):
            nt,accuracy_eval = sess.run([training_op,accuracy], feed_dict={
                    prod_idx: prod_batch,user_idx:user_batch,brand_idx:brand_batch,static_feat:static_batch, 
                    seq_length: seq_length_batch,y:y_batch})
            print(accuracy_eval)
        save_path = saver.save(sess,"/Users/snakepointid/Documents/project/RProj/my code/r相关函数/save/"+save_path+"/test.ckpt")
    elif sessMode=="predict":
        saver.restore(sess,"/Users/snakepointid/Documents/project/RProj/my code/r相关函数/save/"+load_path+"/test.ckpt")
        logits_eval = logits.eval(feed_dict={
                    prod_idx: prod_batch,user_idx:user_batch,brand_idx:brand_batch,static_feat:static_batch, 
                    seq_length: seq_length_batch,y:y_batch})
        print(logits_eval.argmax(1))
    else:
        print("wrong session type")

[2 1 3 8]
